In [3]:
import pandas as pd 

movies_rating = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz")

movies_rating_beau = movies_rating["tconst\taverageRating\tnumVotes"].str.split("\\t", expand=True)

movies_rating_beau.columns = ['ID', 'Note_moyenne', 'Nombre_de_votes']

movies_rating_beau['Note_moyenne'] = movies_rating_beau['Note_moyenne'].astype(float)

movies_rating_beau['Nombre_de_votes'] = movies_rating_beau['Nombre_de_votes'].astype(float)

movies_rating_épuré = movies_rating_beau[movies_rating_beau.Nombre_de_votes > 499]



#contentrating c'est classification d'age ; creator c'est la société de production

df = pd.DataFrame(columns=['ID', 'name', 'alternateName', 'url', 'contentRating', 'datePublished', 'genre', 'actor', 'director', 'creator', 'Origine', 'duration', 'keywords'])



#from pandas.core.dtypes.common import classes_and_not_datetimelike

from bs4 import BeautifulSoup as bs

import requests

from random import seed

import time

import json

import re



#liste des objets inutiles dans le scrap des pages ; je retire aussi le contenu du df qu'on a déjà (les votes)

superflu = ["@context", "@type", "image", "description", "review", "trailer", "aggregateRating"]

#pour tej les images des scénaristes etc

superflu2 = ['@type', 'url']


session_obj = requests.Session()



#on boucle sur chaque film qu'on considère

for ID in movies_rating_épuré['ID'] :

    time.sleep(0.01)

    url_temp = 'https://www.imdb.com/title/'+ID+'/'

    response=session_obj.get(url_temp, headers={"User-Agent": "Mozilla/5.0"})

    html = response.content

    soup = bs(html, "lxml")


#Ça c'est le dictionnaire complet de tout ce qui est aspirable

bloc = soup.find("script", type="application/ld+json").string

dictio = json.loads(bloc)


#on retire dedans ce qui nous intéresse pas

for inutile in superflu :

    dictio.pop(inutile, None)



#on gère les dictionnaires relou pour garder que le nom des personnes
 
if 'actor' in dictio :
    
    for acteur in dictio['actor'] :
    
        for inutile in superflu2 :
    
            acteur.pop(inutile, None)

    for indice, nom in enumerate(dictio['actor']) :
    
            dictio['actor'][indice] = nom['name']
        

if 'director' in dictio :

    for directeur in dictio['director'] :
    
        for inutile in superflu2 :
            
            directeur.pop(inutile, None)
            
        for indice, nom in enumerate(dictio['director']) :

            dictio['director'][indice] = nom['name']
            
            
#pour la société de prod c'est compliqué, ya que une url qui fait ouvrir une page de imdb pro
#d'ailleurs le nom de la société c'est une info qu'on peut pas obtenir autrement
#je pense qu'on peut trifouiller pour l'avoir sans payer mais là faut se contenter de l'url

if 'creator' in dictio :
    
    for createur in dictio['creator'] :

        createur.pop('@type', None)

    for indice, url in enumerate(dictio['creator']) :
    
        dictio['creator'][indice] = url['url']


        
#on ajoute au dictionnaire le pays d'origine ; on le trouve dans la date de sortie

date_sortie_soup = soup.find("a", class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link", href="/title/"+ID+"/releaseinfo?ref_=tt_dt_rdat")

if date_sortie_soup == None :

    date_sortieV2 = "Non renseigné"

    pays = "Non renseigné"

else :
    
    date_sortieV2 = date_sortie_soup.string

    b1 = date_sortieV2.find('(')

    b2 = date_sortieV2.find(')')

    pays = A[b1:b2]

    pays=pays[1:]

    
dictio['Origine'] = pays


print(dictio)

df = df.append(dictio, ignore_index=True)

print(df)

#c'est un test de fabrication de dataframe individuel, pour voir si tout marche bien
#df_temp = pd.DataFrame(list(dictio.values()), index=dictio.keys()).transpose()




FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?